Regrid RAQMS Model following GEMS data using XESMF. 
methods : Bilinear
Average -> regrid averaged data -> plot

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime
import glob
import os
import netCDF4 as nc
import xesmf as xe


In [3]:
#sample dataset check
dataset = xr.open_dataset('/ships22/raqms/lenzen/FV3GFS.9.1.2023/O3.OMPSLPMLSbias.OMPS.NM.CLDFR3.SZA70.O3.thin.25km.and.TROPOMICO.VIIRSAOD.FRAC.em12.phys.1.e-12.phys2/C192/5DEGll/tracer.2023080200.5deg.ll.nc')
no2 = dataset['no2tot']

FileNotFoundError: [Errno 2] No such file or directory: '/ships22/raqms/lenzen/FV3GFS.9.1.2023/O3.OMPSLPMLSbias.OMPS.NM.CLDFR3.SZA70.O3.thin.25km.and.TROPOMICO.VIIRSAOD.FRAC.em12.phys.1.e-12.phys2/C192/5DEGll/tracer.2023080200.5deg.ll.nc'

update Jun 24, the file is not available 

In [9]:
no2.plot() 

NameError: name 'no2' is not defined

In [10]:
no2 = dataset['no2tot']
print(no2.shape)

fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
no2.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=0.01, cbar_kwargs={'label': 'NO2 Total Column'})

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
#ax.set_xlim(95, 128)
#ax.set_ylim(-5, 22)

NameError: name 'dataset' is not defined

In [2]:
files = sorted(glob.glob("/ships22/raqms/lenzen/FV3GFS.9.1.2023/O3.OMPSLPMLSbias.OMPS.NM.CLDFR3.SZA70.O3.thin.25km.and.TROPOMICO.VIIRSAOD.FRAC.em12.phys.1.e-12.phys2/C192/5DEGll/tracer.202308*.nc"))
print(files)

[]


In [3]:
#grouping files by day
daily_files = {}
for f in files:
    filename = os.path.basename(f)
    timestamp = filename.split('.')[1]
    date_str = timestamp[:8]

    if date_str not in daily_files:
        daily_files[date_str] = [f]
    else:
        daily_files[date_str].append(f)
print(daily_files.keys())

dict_keys([])


In [4]:
gems_file = '/ships22/raqms/Satellite/GEMS/L3/NO2/GK2_GEMS_L3_20230802_NO2_GRID-EA_daily_010.nc'
gems_ds = xr.open_dataset(gems_file, group='Geolocation Fields')

gems_lat = gems_ds['Latitude']
gems_lon = gems_ds['Longitude']
target_grid = {'lat': gems_lat, 'lon': gems_lon}

In [5]:
# Define the source grid 
raqms_file = xr.open_dataset(files[0])
raqms_lat = raqms_file['lat'].isel(time=0)
raqms_lon = raqms_file['lon'].isel(time=0)
source_grid = {'lat': raqms_lat, 'lon': raqms_lon}
regridder = xe.Regridder(source_grid, target_grid, method='bilinear', periodic=True)


IndexError: list index out of range

In [6]:
regridder

NameError: name 'regridder' is not defined

In [21]:
# Plotting NO2 for a specific day
# Loop through the dates and plot the NO2 data
dates = [
    '20230801', '20230802', '20230803', '20230804', '20230805',
    '20230806', '20230807', '20230808', '20230809', '20230810',
    '20230811', '20230812', '20230813', '20230814', '20230815',
    '20230816', '20230817', '20230818', '20230819', '20230820',
    '20230821', '20230822', '20230823', '20230824', '20230825',
    '20230826', '20230827', '20230828', '20230829', '20230830',
    '20230831'
]

# Process each day
for date in dates:
    if date not in daily_files or len(daily_files[date]) < 4:
        print(f"Skipping {date}: incomplete data")
        continue

    # Load and sum NO2 for all four time steps
    daily_no2 = []
    for i in range(4):  # 00, 06, 12, 18 UTC
        ds = xr.open_dataset(daily_files[date][i])
        no2 = ds['no2tot'].isel(time=0).values
        daily_no2.append(no2)

    # Compute daily average
    ds_NO2 = np.stack(daily_no2, axis=0)
    no2_daily_avg = ds_NO2.mean(axis=0)
    no2_daily_avg_converted = no2_daily_avg * 100  # convert to 1e16

    # Regrid to GEMS grid
    no2_daily_avg_da = xr.DataArray(
        no2_daily_avg_converted,
        coords={'lat': raqms_lat, 'lon': raqms_lon},
        dims=['grid_yt', 'grid_xt']
    )
    no2_daily_avg_regridded = regridder(no2_daily_avg_da)

    # Plotting
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    mesh = ax.pcolormesh(
        gems_lon, gems_lat, no2_daily_avg_regridded,
        transform=ccrs.PlateCarree(),
        cmap='gist_stern_r',
        vmin=0,
        vmax=3
    )
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_xlim(70, 145)
    ax.set_ylim(-6, 45)
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    ax.set_title(f'RAQMS Daily Avg NO₂ Total Column (Regridded) – {date}')
    plt.colorbar(mesh, orientation='horizontal', label='NO₂ (1e16 molecules/cm²)')

    # Save plot
    os.makedirs("RAQMS_Figures_Regridded", exist_ok=True)
    plt.savefig(f"RAQMS_Figures_Regridded/{date}.png")
    plt.close()
    print(f"{date}")


20230801
20230802
20230803
20230804
20230805
20230806
20230807
20230808
20230809
20230810
20230811
20230812
20230813
20230814
20230815
20230816
20230817
20230818
20230819
20230820
20230821
20230822
20230823
20230824
20230825
20230826
20230827
20230828
20230829
20230830
20230831


In [18]:
print(ds['no2tot'].attrs)

{'units': '10**18 molec'}
